<img src='https://radiant-assets.s3-us-west-2.amazonaws.com/PrimaryRadiantMLHubLogo.png' alt='Radiant MLHub Logo' width='300'/>

# How to use the Radiant MLHub API to browse and download a sample dataset

This Jupyter notebook, which you may copy and adapt for any use, shows basic examples of how to use the API to download labels and source imagery for the LandCoverNet dataset. Full documentation for the API is available at [docs.mlhub.earth](http://docs.mlhub.earth).

We'll show you how to set up your authorization, list collection properties, and retrieve the items (the data contained within them) from those collections.

Each item in our collection is explained in json format compliant with STAC label extension definition.

## Dependencies

This notebook utilizes the [`radiant-mlhub` Python client](https://pypi.org/project/radiant-mlhub/) for interacting with the API. If you are running this notebooks using Binder, then this dependency has already been installed. If you are running this notebook locally, you will need to install this yourself.

See the official [`radiant-mlhub` docs](https://radiant-mlhub.readthedocs.io/) for more documentation of the full functionality of that library.

## Authentication

### Create an API Key

Access to the Radiant MLHub API requires an API key. To get your API key, go to [dashboard.mlhub.earth](https://dashboard.mlhub.earth). If you have not used Radiant MLHub before, you will need to sign up and create a new account. Otherwise, sign in. In the **API Keys** tab, you'll be able to create API key(s), which you will need. *Do not share* your API key with others: your usage may be limited and sharing your API key is a security risk.

### Configure the Client

Once you have your API key, you need to configure the `radiant_mlhub` library to use that key. There are a number of ways to configure this (see the [Authentication docs](https://radiant-mlhub.readthedocs.io/en/latest/authentication.html) for details). 

For these examples, we will set the `MLHUB_API_KEY` environment variable. Run the cell below to save your API key as an environment variable that the client library will recognize.

*If you are running this notebook locally and have configured a profile as described in the [Authentication docs](https://radiant-mlhub.readthedocs.io/en/latest/authentication.html), then you do not need to execute this cell.*


In [1]:
import os
from radiant_mlhub import Dataset
from radiant_mlhub import Collection

os.environ['MLHUB_API_KEY'] = 'YOUR API KEY'

In [ ]:
# List of all Datasets
datasets = Dataset.list()
for dataset in datasets:
    print(dataset)

## Listing Dataset Properties

The following cell makes a request to the API for the properties for the NASA Flood Extent Detection Dataset and prints out a few important properties.

In [3]:
dataset = Dataset.fetch('nasa_floods_v1')

print(f'Title: {dataset.title}')
print(f'DOI: {dataset.doi}')
print(f'Citation: {dataset.citation}')
print('\nCollection IDs and License:')
for collection in dataset.collections:
    print(f'    {collection.id} : {collection.license}')

Title: NASA Flood Extent Detection
DOI: 10.34911/rdnt.ebk43x
Citation: Gahlot, S., Gurung, I., Molthan, A., Maskey, M., & Ramasubramanian, M. (2021) "Flood Extent Data for Machine Learning", Version 1.0, Radiant MLHub. [Date Accessed] https://doi.org/10.34911/rdnt.ebk43x

Collection IDs and License:
    nasa_floods_v1_source : CC-BY-4.0
    nasa_floods_v1_labels : CC-BY-4.0


## Downloading Assets

> **NOTE:** If you are running these notebooks using Binder these resources will be downloaded to the remote file system that the notebooks are running on and **not to your local file system.** If you want to download the files to your machine, you will need to clone the repo and run the notebook locally.

This next cell will call the dataset download function with a filter specified which only downloads the `raster_label` and `VV` source assets within the `nasa_floods_v1` collection. For more information about filtering downloads, reference the [collection and asset key filtering method in the Python client documenation](https://radiant-mlhub.readthedocs.io/en/latest/datasets.html#filter-by-collection-and-asset-keys).

In [ ]:
aoi = {
  "type": "Feature",
  "properties": {},
  "geometry": {
    "type": "Polygon",
    "coordinates": [
      [
        [
          88.79150390625,
          22.705255477207526
        ],
        [
          93.49365234375,
          22.705255477207526
        ],
        [
          93.49365234375,
          27.068909095463365
        ],
        [
          88.79150390625,
          27.068909095463365
        ],
        [
          88.79150390625,
          22.705255477207526
        ]
      ]
    ]
  }
}

In [ ]:
from dateutil.parser import parse
my_start_date=parse("2017-06-01T00:00:00+0000")
my_end_date=parse("2017-06-30T00:00:00+0000")

asset_filter = dict(
    nasa_floods_v1_labels=['raster_label'],
    nasa_floods_v1_source=['VV']
)

In [ ]:
dataset.download(intersects=aoi, datetime=(my_start_date, my_end_date), collection_filter=asset_filter)

### Download All Assets

The following cell uses the `dataset.download` function to download the `nasa_floods_v1` dataset to the current working directory.

In [ ]:
dataset.download()